<h1>FINAL PROJECT</h1>
<h4>Analisis Big Data A</h4>

<h5>1. Import Libraries</h5>

In [1]:
from bs4 import BeautifulSoup
from requests import get
import pandas as pd
from time import sleep
from time import time
from random import randint

<h5>2. Proses Crawl</h5>
<p>Variabel list yang akan dijadikan kolom tabel</p>

In [2]:
names = []
years = []
imdb_ratings = []
metascores = []
votes = []
run_time=[]
directors = []
genres = []
income = []

start_time = time()
requests = 0

Indexing URL

In [3]:
j=[1,51,101,151,201,251,301,351,401,451,501,551,601,651,701,751,801,851,901,951,1001,1051,1101]
pages = [str(i) for i in j[:2]]# hanya crawling dari film 1 - 1150
years_url = [str(i) for i in range(2010,2020)] #dari tahun 2010 - 2019

<p>Proses Crawl Data dari URL imdb berdasarkan jumlah vote</p>
<p>Data yang diambil dari film yang tidak memiliki 'metascore'</p>

In [4]:
for year_url in years_url:
    for page in pages:       
        url = 'http://www.imdb.com/search/title?release_date='+year_url+'-01-01,'+year_url+'-12-31&sort=num_votes,desc&start='+page+'&ref_=adv_nxt'
        print(url)
        response = get(url)
        
        #pause loop
        sleep(randint(10,50))
        
        #increase request number
        requests += 1
        elapsed_time = time() - start_time
        print('Request: {}; Frequency: {} requests/s'.format(requests, requests/elapsed_time))
        #print(response.text)
        soup = BeautifulSoup(response.text, 'html.parser')

        movie_containers = soup.find_all('div',class_='lister-item mode-advanced')
    
        # ambil data dari tiap container
        for container in movie_containers:
        # jika ada rating metascoare:
            if container.find('div', class_ = 'ratings-metascore') is not None:
                # judul
                name = container.h3.a.text
                names.append(name)
                # tahun
                year = container.h3.find('span', class_ = 'lister-item-year').text
                years.append(year)
                # IMDB rating
                imdb = float(container.strong.text)
                imdb_ratings.append(imdb)
                # Metascore
                m_score = container.find('span', class_ = 'metascore').text
                metascores.append(int(m_score))
                # jumlah vote
                vote = container.find('span', attrs = {'name':'nv'})['data-value']
                votes.append(int(vote))
                 #Run Time
                r_time = container.find('span', class_= 'runtime').text
                run_time.append(int(r_time[0:3]))
                #Directors
                director = container.find('p',class_="").text
                directors.append(director)
                #Genre
                gen = container.find('span', class_='genre').text
                genres.append(gen)
                #Gross
                gross = container.find('p',class_='sort-num_votes-visible').text
                income.append(gross)   

http://www.imdb.com/search/title?release_date=2010-01-01,2010-12-31&sort=num_votes,desc&start=1&ref_=adv_nxt
Request: 1; Frequency: 0.019766688917110906 requests/s
http://www.imdb.com/search/title?release_date=2010-01-01,2010-12-31&sort=num_votes,desc&start=51&ref_=adv_nxt
Request: 2; Frequency: 0.021346511553512897 requests/s
http://www.imdb.com/search/title?release_date=2011-01-01,2011-12-31&sort=num_votes,desc&start=1&ref_=adv_nxt
Request: 3; Frequency: 0.02276168107233864 requests/s
http://www.imdb.com/search/title?release_date=2011-01-01,2011-12-31&sort=num_votes,desc&start=51&ref_=adv_nxt
Request: 4; Frequency: 0.023611869491895494 requests/s


Membuat data frame 'movie_rating' berdasarkan kolom yang telah di dapat dalam proses crawl

In [5]:
movie_ratings = pd.DataFrame({'movie': names,
'year': years,
'imdb': imdb_ratings,
'metascore': metascores,
'votes': votes,
'duration (min)':run_time,
'directors':directors,
'genres' : genres,
'income' : income
})

In [6]:
movie_ratings.head()

,movie,year,imdb,metascore,votes,duration (min),directors,genres,income
0,Inception,(2010),8.8,74,1861169,148,\n Director:\nChristopher Nolan\n| \n St...,"\nAction, Adventure, Sci-Fi","\nVotes:\n1,861,169\n| Gross:\n$292.58M\n"
1,Shutter Island,(2010),8.1,63,1017828,138,\n Director:\nMartin Scorsese\n| \n Star...,"\nMystery, Thriller","\nVotes:\n1,017,828\n| Gross:\n$128.01M\n"
2,Toy Story 3,(2010),8.3,92,695286,103,\n Director:\nLee Unkrich\n| \n Stars:\n...,"\nAnimation, Adventure, Comedy","\nVotes:\n695,286\n| Gross:\n$415.00M\n"
3,Iron Man 2,(2010),7.0,57,666672,124,\n Director:\nJon Favreau\n| \n Stars:\n...,"\nAction, Adventure, Sci-Fi","\nVotes:\n666,672\n| Gross:\n$312.43M\n"
4,Black Swan,(2010),8.0,79,653913,108,\n Director:\nDarren Aronofsky\n| \n Sta...,"\nDrama, Thriller","\nVotes:\n653,913\n| Gross:\n$106.95M\n"


<h5>Proses Celaning data 1</h5>

In [8]:
movie_ratings['directors'].head()

0    \n    Director:\nChristopher Nolan\n| \n    St...
1    \n    Director:\nMartin Scorsese\n| \n    Star...
2    \n    Director:\nLee Unkrich\n| \n    Stars:\n...
3    \n    Director:\nJon Favreau\n| \n    Stars:\n...
4    \n    Director:\nDarren Aronofsky\n| \n    Sta...
Name: directors, dtype: object

In [9]:
#Clean Directors
tes = movie_ratings['directors']
dire = []
for x in range(len(tes)):
    k=tes[x]
    y=''
    for i in k[15:]:
        j=i
        if j != '|':
            y=y+j
        else : break
    dire.append(y)

In [15]:
dire[:5]

['Christopher Nolan\n',
 'Martin Scorsese\n',
 'Lee Unkrich\n',
 'Jon Favreau\n',
 'Darren Aronofsky\n']

In [14]:
movie_ratings['income'].head()

0    \nVotes:\n1,861,169\n| Gross:\n$292.58M\n
1    \nVotes:\n1,017,828\n| Gross:\n$128.01M\n
2      \nVotes:\n695,286\n| Gross:\n$415.00M\n
3      \nVotes:\n666,672\n| Gross:\n$312.43M\n
4      \nVotes:\n653,913\n| Gross:\n$106.95M\n
Name: income, dtype: object

In [18]:
#Clean Income
tes = income
inc = []
for x in range(len(tes)):
    k=tes[x]
    y=''
    dol='0'
    for i in k:
        if i == '$' : 
            dol=i
        if dol == '$' and i!='$':
            y=y+i
        if i=='M' : break
    if y == '':y='0M'
    inc.append(y)

In [22]:
inc

['292.58M',
 '128.01M',
 '415.00M',
 '312.43M',
 '106.95M',
 '217.58M',
 '138.80M',
 '96.96M',
 '48.07M',
 '251.51M',
 '295.98M',
 '200.82M',
 '334.19M',
 '31.49M',
 '58.40M',
 '18.34M',
 '92.19M',
 '93.62M',
 '103.07M',
 '100.54M',
 '172.06M',
 '171.24M',
 '118.31M',
 '94.84M',
 '90.38M',
 '163.21M',
 '54.01M',
 '90.76M',
 '105.27M',
 '77.22M',
 '119.22M',
 '300.53M',
 '162.00M',
 '148.42M',
 '67.63M',
 '52.00M',
 '26.59M',
 '81.56M',
 '9.74M',
 '76.42M',
 '21.15M',
 '238.37M',
 '32.37M',
 '88.76M',
 '60.97M',
 '0.22M',
 '50.29M',
 '60.13M',
 '15.54M',
 '176.59M',
 '98.71M',
 '63.15M',
 '1.03M',
 '131.56M',
 '104.39M',
 '2.43M',
 '19.07M',
 '80.01M',
 '6.53M',
 '6.86M',
 '6.35M',
 '33.58M',
 '35.05M',
 '20.81M',
 '31.58M',
 '53.37M',
 '67.06M',
 '110.49M',
 '12.13M',
 '23.98M',
 '39.12M',
 '2.70M',
 '148.38M',
 '61.98M',
 '13.79M',
 '0.13M',
 '23.24M',
 '52.47M',
 '84.75M',
 '40.17M',
 '73.03M',
 '4.02M',
 '27.76M',
 '32.75M',
 '0.21M',
 '35.61M',
 '25.92M',
 '53.03M',
 '63.08M',
 '43

<p>Membuat data frame 'movie_rating' baru berdasarkan kolom baru yang telah di dapat dalam proses crawl</p>

In [23]:
#definisi movie rating kedua
movie_ratings = pd.DataFrame({'movie': names,
'year': years,
'imdb': imdb_ratings,
'metascore': metascores,
'votes': votes,
'duration (min)':run_time,
'directors':dire,
'genres' : genres,
'income':inc
})

In [ ]:
#Gabung Kolom
movie_ratings = movie_ratings[['movie', 'year', 'imdb', 'metascore', 'votes', 'duration (min)','directors','genres','income']]
print(movie_ratings.head())

In [24]:
movie_ratings.head()

,movie,year,imdb,metascore,votes,duration (min),directors,genres,income
0,Inception,(2010),8.8,74,1861169,148,Christopher Nolan\n,"\nAction, Adventure, Sci-Fi",292.58M
1,Shutter Island,(2010),8.1,63,1017828,138,Martin Scorsese\n,"\nMystery, Thriller",128.01M
2,Toy Story 3,(2010),8.3,92,695286,103,Lee Unkrich\n,"\nAnimation, Adventure, Comedy",415.00M
3,Iron Man 2,(2010),7.0,57,666672,124,Jon Favreau\n,"\nAction, Adventure, Sci-Fi",312.43M
4,Black Swan,(2010),8.0,79,653913,108,Darren Aronofsky\n,"\nDrama, Thriller",106.95M


In [25]:
#Proses ke CSV
movie_ratings.loc[:, 'year'] = movie_ratings['year'].str[-5:-1].astype(str)
movie_ratings.loc[:, 'directors'] = movie_ratings['directors'].str[:-1]
movie_ratings.loc[:, 'genres'] = movie_ratings['genres'].str[0:]
movie_ratings.describe().loc[['min', 'max'], ['imdb', 'metascore']]
movie_ratings.loc[:, 'income'] = movie_ratings['income'].str[:-1].astype(float)

In [26]:
#memasukkan nilai 100 pada data kosong di kolom income
for s in range(len(movie_ratings['income'])):
    if movie_ratings['income'][s] == 0 : movie_ratings['income'][s] = 0.01

In [27]:
movie_ratings.head()

,movie,year,imdb,metascore,votes,duration (min),directors,genres,income
0,Inception,2010,8.8,74,1861169,148,Christopher Nolan,"\nAction, Adventure, Sci-Fi",292.58
1,Shutter Island,2010,8.1,63,1017828,138,Martin Scorsese,"\nMystery, Thriller",128.01
2,Toy Story 3,2010,8.3,92,695286,103,Lee Unkrich,"\nAnimation, Adventure, Comedy",415.00
3,Iron Man 2,2010,7.0,57,666672,124,Jon Favreau,"\nAction, Adventure, Sci-Fi",312.43
4,Black Swan,2010,8.0,79,653913,108,Darren Aronofsky,"\nDrama, Thriller",106.95


In [ ]:
movie_ratings.to_csv("movie_ratings10-14.csv")
movie_ratings.head(10)